# Crack Detection using Drone Images

This notebook processes images to detect cracks using various image processing techniques.

In [1]:
!pip install opencv-python-headless pillow matplotlib scikit-image

In [2]:
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Load image
image_path = 'download.jpeg'  # Adjust the image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [4]:
# Noise reduction
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

In [5]:
# Adaptive histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(blurred_image)

In [6]:
# GLCM
glcm = greycomatrix(enhanced_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = greycoprops(glcm, 'contrast')[0, 0]
dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
energy = greycoprops(glcm, 'energy')[0, 0]
correlation = greycoprops(glcm, 'correlation')[0, 0]
ASM = greycoprops(glcm, 'ASM')[0, 0]

print(f'GLCM - Contrast: {contrast}, Dissimilarity: {dissimilarity}, Homogeneity: {homogeneity}, Energy: {energy}, Correlation: {correlation}, ASM: {ASM}')

In [7]:
# LBP
radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(enhanced_image, n_points, radius, method='uniform')

In [8]:
# Edge detection
edges = cv2.Canny(enhanced_image, 100, 200)

In [9]:
# Display images
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1), plt.imshow(image, cmap='gray'), plt.title('Original Image')
plt.subplot(2, 2, 2), plt.imshow(blurred_image, cmap='gray'), plt.title('Blurred Image')
plt.subplot(2, 2, 3), plt.imshow(enhanced_image, cmap='gray'), plt.title('Enhanced Image')
plt.subplot(2, 2, 4), plt.imshow(edges, cmap='gray'), plt.title('Edges')
plt.show()